In [1]:
from row_col_matrix import MojoMatrix
vals = List[Float64](1,2,3,4,5,6,7,8,9,10,11,12)
M = MojoMatrix(4,3,vals)
print(M)
print("Memory layout:")
M.print_linear()

[[1.0  2.0  3.0]
 [4.0  5.0  6.0]
 [7.0  8.0  9.0]
 [10.0  11.0  12.0]]
Matrix: 4x3 | DType:float64 | Row Major

Memory layout:
[1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0]



In [1]:
from row_col_matrix import MojoMatrix
vals = List[Float64](1,2,3,4,5,6,7,8,9,10,11,12)
M = MojoMatrix(4,3,vals)
mm_col_major = M.to_colmajor()
print(mm_col_major)
print("Memory layout:")
mm_col_major.print_linear()

[[1.0  2.0  3.0]
 [4.0  5.0  6.0]
 [7.0  8.0  9.0]
 [10.0  11.0  12.0]]
Matrix: 4x3 | DType:float64 | Column Major

Memory layout:
[1.0 4.0 7.0 10.0 2.0 5.0 8.0 11.0 3.0 6.0 9.0 12.0]



In [1]:
from row_col_matrix import MojoMatrix
vals = List[Float64](1,2,3,4,5,6,7,8,9,10,11,12)
M = MojoMatrix(4,3,vals)
mm_col_major = M.to_colmajor()
print("Matrix in column-major order:")
print(mm_col_major)
print("Memory layout:")
mm_col_major.print_linear()
print("Transposed matrix:")
print(M.transpose())
print("Memory layout:")
M.transpose().print_linear()

Matrix in column-major order:
[[1.0  2.0  3.0]
 [4.0  5.0  6.0]
 [7.0  8.0  9.0]
 [10.0  11.0  12.0]]
Matrix: 4x3 | DType:float64 | Column Major

Memory layout:
[1.0 4.0 7.0 10.0 2.0 5.0 8.0 11.0 3.0 6.0 9.0 12.0]

Transposed matrix:
[[1.0  4.0  7.0  10.0]
 [2.0  5.0  8.0  11.0]
 [3.0  6.0  9.0  12.0]]
Matrix: 3x4 | DType:float64 | Row Major

Memory layout:
[1.0 4.0 7.0 10.0 2.0 5.0 8.0 11.0 3.0 6.0 9.0 12.0]



In [1]:
from row_col_matrix import MojoMatrix
import benchmark
from random import random_si64
from benchmark import Unit, keep
from python import Python
from time import sleep
from testing import assert_almost_equal
from algorithm import vectorize

fn mojo_to_py_list(list: List[Int]) raises -> PythonObject:
    var py_list = PythonObject([])
    for idx in list:
        py_list.append(idx[])
    return py_list

fn print_numpy_compare_norm(np_mat: PythonObject, mat: MojoMatrix, mean_idx: List[Int]) raises -> Float64:
    var np = Python.import_module("numpy")
    var np_mean: PythonObject
    var mat_mean: PythonObject
    if len(mean_idx)==0:
        np_mean = np_mat.mean(axis=0)
        mat_mean = mat.mean(mean_idx).to_numpy()
    else:
        np_mean = np_mat.take(mojo_to_py_list(mean_idx),axis=1).mean(axis=0)
        mat_mean = mat.mean(mean_idx).to_numpy()       
    return np.linalg.norm(np_mean - mat_mean).to_float64()

fn bench_row_major(np_mat: PythonObject, mean_idx: List[Int]) raises -> Float64:
    var mat = MojoMatrix.from_numpy(np_mat)
    @parameter
    fn benchmark_fn():
        var m = mat.mean(mean_idx)
        keep(m._matPtr)

    var report_row_major = benchmark.run[benchmark_fn]()

    print("Row-major accuracy check vs. NumPy. (2-Norm of difference):",str(print_numpy_compare_norm(np_mat, mat, mean_idx)))
    return report_row_major.mean(Unit.ms)

fn bench_col_major(np_mat: PythonObject, mean_idx: List[Int]) raises -> Float64:
    var mat = MojoMatrix.from_numpy(np_mat).to_colmajor()
    @parameter
    fn benchmark_fn():
        var m = mat.mean(mean_idx)
        keep(m._matPtr)

    var report_col_major = benchmark.run[benchmark_fn]()

    print("Column-major accuracy check vs. NumPy. (2-Norm of difference):",str(print_numpy_compare_norm(np_mat, mat, mean_idx)))
    return report_col_major.mean(Unit.ms)

fn bench_numpy_row_major(np_mat: PythonObject, mean_idx: PythonObject) raises -> Float64:
    Python.add_to_path(".")
    var numpy_mean = Python.import_module("python_mean_bench")
    return numpy_mean.row_major_mean(np_mat, mean_idx).to_float64()

fn bench_numpy_col_major(np_mat: PythonObject, mean_idx: PythonObject) raises -> Float64:
    Python.add_to_path(".")
    var numpy_mean = Python.import_module("python_mean_bench")
    return numpy_mean.col_major_mean(np_mat, mean_idx).to_float64()

np = Python.import_module("numpy")
np_mat = np.random.rand(10000,10000)

mean_idx = List[Int]()
for i in range(1000):
    mean_idx.append(int(random_si64(0,10000)))

row_major_mean_time = bench_row_major(np_mat, mean_idx)
col_major_mean_time = bench_col_major(np_mat, mean_idx)

numpy_row_major_mean_time = bench_numpy_row_major(np_mat, mojo_to_py_list(mean_idx))
numpy_col_major_mean_time = bench_numpy_col_major(np_mat, mojo_to_py_list(mean_idx))

print()
print("========== Execution time ==========")
print()
print("Mojo row-major mean time (ms):",row_major_mean_time)
print("Mojo column-major mean time (ms):",col_major_mean_time)
print()
print("NumPy row-major mean time (ms):",numpy_row_major_mean_time)
print("NumPy column-major mean time (ms):",numpy_col_major_mean_time)
print()
print("========== Speedups ==========")
print()
print("Col-major vs. Row-Major")
print("Mojo col-major vs. Mojo row-major:",str(row_major_mean_time/col_major_mean_time)[:6],end='x\n')
print("NumPy col-major vs. NumPy row-major (default):",str(numpy_row_major_mean_time/numpy_col_major_mean_time)[:6],end='x\n')
print()
print("Mojo vs. NumPy")
print("Mojo col-major vs. NumPy row-major (default):",str(numpy_row_major_mean_time/col_major_mean_time)[:6],end='x\n')
print("Mojo col-major vs. NumPy col-major:",str(numpy_col_major_mean_time/col_major_mean_time)[:6],end='x\n')

Row-major accuracy check vs. NumPy. (2-Norm of difference): 4.8095599519507898e-14
Column-major accuracy check vs. NumPy. (2-Norm of difference): 4.8095599519507898e-14

========== Execution time ==========

Mojo row-major mean time (ms): 39.034610169491522
Mojo column-major mean time (ms): 1.7911874074074075

NumPy row-major mean time (ms): 47.496916609816253
NumPy column-major mean time (ms): 8.1102000083774328

========== Speedups ==========

Col-major vs. Row-Major
Mojo col-major vs. Mojo row-major: 21.792x
NumPy col-major vs. NumPy row-major (default): 5.8564x

Mojo vs. NumPy
Mojo col-major vs. NumPy row-major (default): 26.517x
Mojo col-major vs. NumPy col-major: 4.5278x
